In [5]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from sqlalchemy import create_engine
from sqlalchemy import text

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix

In [6]:
engine = create_engine('sqlite:///../data/titanic.db')
sqlite_connection = engine.connect()
pd.read_sql(text('SELECT * FROM sqlite_schema WHERE type="table"'), con=sqlite_connection)

,type,name,tbl_name,rootpage,sql
0,table,tbl_passengers,tbl_passengers,2,"CREATE TABLE tbl_passengers (\n\tpid BIGINT, \..."
1,table,tbl_targets,tbl_targets,35,"CREATE TABLE tbl_targets (\n\tpid BIGINT, \n\t..."


In [7]:
np.random.seed(42)

df = pd.read_sql(text('SELECT * FROM tbl_passengers'), con=sqlite_connection)

targets = pd.read_sql(text('SELECT * FROM tbl_targets'), con=sqlite_connection)

# df, targets = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

# parch = Parents/Children, sibsp = Siblings/Spouses
df['family_size'] = df['parch'] + df['sibsp']
df['is_alone'] = [1 if family_size==1 else 0 for family_size in df['family_size']]

df['title'] = [name.split(',')[1].split('.')[0].strip() for name in df['name']]
rare_titles = {k for k,v in Counter(df['title']).items() if v < 10}
df['title'] = ['rare' if title in rare_titles else title for title in df['title']]

df = df[[
    'pclass', 'sex', 'age', 'ticket', 'family_size',
    'fare', 'embarked', 'is_alone', 'title'
]]

targets = [int(v) for v in targets['is_survived']]
X_train, X_test, y_train, y_test = train_test_split(df, targets, stratify=targets, test_size=0.2)

In [8]:
df[:3]

,pclass,sex,age,ticket,family_size,fare,embarked,is_alone,title
0,1.0,female,29.0000,24160,0.0,211.3375,S,0,Miss
1,1.0,male,0.9167,113781,3.0,151.5500,S,0,Master
2,1.0,female,2.0000,113781,3.0,151.5500,S,0,Miss


In [9]:
X_train_categorical = X_train[['embarked', 'sex', 'pclass', 'title', 'is_alone']]
X_test_categorical = X_test[['embarked', 'sex', 'pclass', 'title', 'is_alone']]

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(X_train_categorical)
X_train_categorical_one_hot = one_hot_encoder.transform(X_train_categorical)
X_test_categorical_one_hot = one_hot_encoder.transform(X_test_categorical)

/opt/anaconda3/envs/SQL/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
X_train_numerical = X_train[['age', 'fare', 'family_size']]
X_test_numerical = X_test[['age', 'fare', 'family_size']]
knn_imputer = KNNImputer(n_neighbors=5).fit(X_train_numerical)
X_train_numerical_imputed = knn_imputer.transform(X_train_numerical)
X_test_numerical_imputed = knn_imputer.transform(X_test_numerical)

In [11]:
robust_scaler = RobustScaler().fit(X_train_numerical_imputed)
X_train_numerical_imputed_scaled = robust_scaler.transform(X_train_numerical_imputed)
X_test_numerical_imputed_scaled = robust_scaler.transform(X_test_numerical_imputed)

In [12]:
X_train_processed = np.hstack((X_train_categorical_one_hot, X_train_numerical_imputed_scaled))
X_test_processed = np.hstack((X_test_categorical_one_hot, X_test_numerical_imputed_scaled))

In [13]:
model = LogisticRegression(random_state=0).fit(X_train_processed, y_train)
y_train_estimation = model.predict(X_train_processed)
y_test_estimation = model.predict(X_test_processed)

In [14]:
cm_train = confusion_matrix(y_train, y_train_estimation)

In [15]:
cm_test = confusion_matrix(y_test, y_test_estimation)

In [16]:
cm_train

array([[553,  94],
       [107, 293]])

In [17]:
cm_test

array([[142,  20],
       [ 22,  78]])

In [23]:
rare_titles

{'Capt',
 'Col',
 'Don',
 'Dona',
 'Dr',
 'Jonkheer',
 'Lady',
 'Major',
 'Mlle',
 'Mme',
 'Ms',
 'Rev',
 'Sir',
 'the Countess'}